# Text Mining – Project 

## "Metaphor Detection Using Distributional Semantics"

In [ ]:
# Full name, starting with your LASTNAME and then  FIRSTNAME
ProjectMembers = "Ahmed Jasim"

email = "ahmed05@ads.uni-passau.de"

In [1]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context
    
    
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [20]:
try:
    import pandas as pd
except ModuleNotFoundError:
    !pip install pandas
    
try:
    import numpy as np
except ModuleNotFoundError:
    !pip install numpy
    
try:
    import re
except ModuleNotFoundError:
    !pip install re

try:
    import sklearn
except ModuleNotFoundError:
    !pip install scikit-learn   

from nltk.stem import WordNetLemmatizer
from nltk import FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB

## Downloading the Metaphor AN Dataset

In [ ]:
# The annotated data set is publicly available so it needs to be downloading to the local machine 
# http://bit.ly/1TQ5czN

## Setting Up the corpus

In [3]:
from nltk.corpus import brown
from nltk.corpus import stopwords

In [4]:
word_lemmatizer = WordNetLemmatizer()
stoplist = stopwords.words('english')

# Feteching the phrases from CSV file
# Need to add the location of Metaphor AN phrases on local machine
review = pd.read_csv('/home/airwolf/Documents/TMP18_9_Distributional_Semantic_Model_Metaphor/AN-phrase-annotations.csv')
# Converting the csv file into Data Frame:
toDTM = CountVectorizer(lowercase = False)
review_df = pd.DataFrame(review)
review_list = review.values.tolist()


In [5]:
# Metaphor Phrases in list
metaphorAN_phrases = []
for review_list_inner in review_list: 
    tempList = []
    for word in review_list_inner[:2]:
        tempList.append("".join(re.split(r'_\w$', word)[:1]))
    metaphorAN_phrases.append(" ".join(tempList))
    
# AnnotationTags in list
AnnotationTags = []
for review_list_tag in review_list:
    AnnotationTags.append(review_list_tag[2])

# Final Metaphors with AnnotationTags in list
metaphorANPhrases_WithTags = (np.column_stack((metaphorAN_phrases,AnnotationTags))).tolist()

## Finding adjective noun phrases in the corpus and their respective Sentences

In [6]:
def getAdjNounPhraseAndSent(tagged_sents):    
    adjNounPhrase_and_Sents = []
    for sent in tagged_sents:
        # Tagged Sentence Without Stop Words
        sentWithNoStopWords = [word_tuple for word_tuple in sent if word_tuple[0] not in stoplist]
        temp_phrase_list = []
        adjCount = 0
        nounCount = 0
        lemmatized_sent = []
        # Lemmatization of Corpus sentences with respect to noun, adjective and verbs
        pos_list = ["a","n","v"]
        for word_tuple in sentWithNoStopWords:
            if word_tuple[1].lower()[:1] in pos_list:
                lemmatized_sent.append((word_lemmatizer.lemmatize(word_tuple[0],word_tuple[1].lower()[:1]),word_tuple[1]))
            else:
                lemmatized_sent.append(word_tuple)
        # Extracting the Phrases that have occurence of Noun and Adjective
        for word in lemmatized_sent:
            if word[1] == "NOUN":
                nounCount += 1
                temp_phrase_list.append(word[0])

            elif word[1] == "ADJ":
                adjCount += 1
                temp_phrase_list.append(word[0])

        if nounCount >= 1 and adjCount >= 1:
            adjNounPhrase_and_Sents.append((" ".join(temp_phrase_list),lemmatized_sent))
    # Extracting Adjective Noun Phrases and their Sentences
    return (adjNounPhrase_and_Sents)
                

### Finding the Common phrases between Metaphor AN Phrases and Adjective Noun Phrases of the Corpus

In [7]:
# finding Metaphor AN Anotation in adjNounPhrases
def findAdjNounPhrases_Common_in_MetaphorAN_and_adjNounPhrases_of_Corpus(adjNounPhraseSent_tuple,MetaphorAN_with_Tags):
    commonAdjNounPhrase = []
    contextWords = []
    phraseAndSents_dict = {}
    finalMetaphorANSents = []
    finalAnnotatedTags = []
    for phraseAndTag in MetaphorAN_with_Tags:
        # splitting the Metaphor AN Phrases and loading them into set
        phraseAN_set = set(phraseAndTag[0].split())
        commonSents = []
        countOfMetaphorPhraseOccurence = 0
        for cPhrase,cSents in adjNounPhraseSent_tuple:
            # splitting the Adjective Noun Phrases and loading them into set
            cPhrase_set = set(cPhrase.split())
            # sentence without pos tags
            sentsWithOutTags = " ".join([wordTuple[0].lower() for wordTuple in cSents]) 
            # Common Phrase between Metaphor AN and Phrases of Corpus
            commonWords = phraseAN_set & cPhrase_set
            if len(commonWords) == len(phraseAN_set):
                countOfMetaphorPhraseOccurence += 1
                commonSents.append(sentsWithOutTags)
                # removing the Metaphor word from the phrase
                filteredWord = ((cPhrase.replace(phraseAndTag[0].split()[0],"")).replace(phraseAndTag[0].split()[1],""))
                 # removing the unwanted white spaces
                words = (" ".join(filteredWord.split())).split()
                # context words in the phrases
                contextWords.extend(words) 
                
        if countOfMetaphorPhraseOccurence >= 1:
            commonAdjNounPhrase.append(phraseAndTag[0])
            # metaphor Phrases with their respective sentences 
            phraseAndSents_dict[phraseAndTag[0]] = commonSents
            finalMetaphorANSents.append(" ".join(commonSents))
            # Final Annotated Tags
            finalAnnotatedTags.append(phraseAndTag[1])


    context = [word for word in contextWords if len(word) > 3]
    
    return (finalMetaphorANSents,finalAnnotatedTags,context,commonAdjNounPhrase,phraseAndSents_dict)

## Start of the Project......

In [8]:
# Start------------------------
cummulative_Tagged_Sents = []
# brown Tagged Sentences 
brown_tagged_sents = [sents for category in brown.categories() for sents in brown.tagged_sents(categories=category, tagset = 'universal')]
## If needed more sentences then tree bank dataset can also serve more pos tagged sentences.
# treebank_tagged_sents = nltk.corpus.treebank.tagged_sents(tagset='universal')

cummulative_Tagged_Sents.extend(brown_tagged_sents)
# for innerList in treebank_tagged_sents:
#     cummulative_Tagged_Sents.append(innerList)
# brown_tagged_sents = [sents for sents in brown.tagged_sents(categories="news", tagset = 'universal')]
adjNounPhraseAndSent_tuple =  getAdjNounPhraseAndSent(cummulative_Tagged_Sents)


### Extracting the Common Sentences, Annotated Tags and Context

In [9]:
commonSentencesAnnotatedTagsAndContext = findAdjNounPhrases_Common_in_MetaphorAN_and_adjNounPhrases_of_Corpus(adjNounPhraseAndSent_tuple,metaphorANPhrases_WithTags)


In [11]:
finalMetaphorSent = commonSentencesAnnotatedTagsAndContext[0]
finalMetaphorTags = commonSentencesAnnotatedTagsAndContext[1]
contextWords = commonSentencesAnnotatedTagsAndContext[2]
commonAdjNounPhrase = commonSentencesAnnotatedTagsAndContext[3]
contextTerms = []

# Finding most common terms in context
fdist = FreqDist(contextWords)
words = fdist.most_common(800)

for word_tuple in words:
    contextTerms.append(word_tuple[0])


###  Processing the Metaphor sentences (data) and Context Terms using TF-IDF 

In [15]:
vectorizer = TfidfVectorizer()
vectorizer = vectorizer.fit(contextTerms)
mat = vectorizer.transform(finalMetaphorSent)

In [16]:
cfd = pd.DataFrame(mat.todense(),columns = vectorizer.get_feature_names())
cfd

,account,acid,action,additional,advantage,adventure,agent,airport,airy,alice,...,word,work,world,wretched,writer,yard,year,yesterday,york,young
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.268343,0.0,0.000000,0.268343
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.273222,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.447214,0.0,0.000000,0.000000
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.353553,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.353553,0.353553
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000


### Feature Extraction using Train Test Split

In [17]:
y_labels = finalMetaphorTags
X_train, X_test, y_train, y_test = train_test_split(cfd, y_labels, test_size=0.29, random_state=38)

In [54]:
clf = BernoulliNB()
clf.fit(X_train, y_train)
BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)
print(clf.predict(X_test[1:10]))
clf.score(X_test,y_test)

['y' 'y' 'n' 'y' 'y' 'y' 'n' 'n' 'y']


0.8274932614555256

In [71]:
clf = GaussianNB()
clf.fit(X_train, y_train)
GaussianNB(priors=None, var_smoothing=1e-09)
print(clf.predict(list(np.arange(791).reshape(1,791))))
clf.score(X_test,y_test)
# clf_pf = GaussianNB()
# clf_pf.partial_fit(X_train, y_train, np.unique(y_train))
# GaussianNB(priors=None, var_smoothing=1e-09)
# print(clf_pf.predict(list(np.arange(791).reshape(1,791))))
# clf.score(X_test,y_test)


['n']


0.7358490566037735

In [53]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
clf.fit(X_train, y_train)  
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)
# print(clf.feature_importances_)
print(clf.predict(list(np.arange(791).reshape(1,791))))
clf.score(X_test,y_test)

['y']


0.6307277628032345

## Classifier Training and Evaluation

### MultinomialNaiveBayes Classifer 

In [87]:
classifier_mul = MultinomialNB().fit(X_train,y_train)
classifier_mul.score(X_test,y_test)

0.8409703504043127

### Decision Tree Classifier

In [88]:
classifier_dt = DecisionTreeClassifier().fit(X_train,y_train)
classifier_dt.score(X_test,y_test)

0.7466307277628033

### Using Cross Validation KFold and RepeatedKFold

In [91]:
classifier_Multinomial = MultinomialNB()
classifier_LogisticRegression = LogisticRegression(solver='lbfgs', multi_class='multinomial', random_state=1)
classifier_RandomForestClassifier = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
classifier_DecisionTreeClassifier = DecisionTreeClassifier(max_depth=None, min_samples_split=2, random_state=0)

kf = KFold(n_splits=10)
random_state = 300
rkf = RepeatedKFold(n_splits=10, n_repeats=2, random_state=random_state)

scoring = ['precision_macro', 'recall_macro','accuracy','f1_macro']
scores = cross_validate(classifier_Multinomial, cfd, y_labels, cv=rkf,scoring=scoring)


### Accuracy, Precision, Recall and F1_Score

In [92]:
print("Accuracy : ",np.average(scores['test_accuracy']))
print("Precision : ",np.average(scores['test_precision_macro']))
print("Recall : ",np.average(scores['test_recall_macro']))
print("F1_macro : ",np.average(scores['test_f1_macro']))

Accuracy :  0.8092919537401576
Precision :  0.8107667287161611
Recall :  0.8083990194991679
F1_macro :  0.8076070736175952
